# This file works with FILAMENT ERUPTION and October 2013 Track information 

In [11]:
#all imports 
from sunpy.net import hek
from sunpy.net import hek2vso
import matplotlib.pyplot as plt
%matplotlib inline
import shapely
from shapely.wkt import dumps, loads
import numpy as np
import pandas as pd
from shapely.geometry.polygon import LinearRing, Polygon
from datetime import timedelta
import datetime
from sunpy.time import *
from dateutil.parser import parse
import matplotlib.cm as cm
import matplotlib as mpl
from operator import itemgetter
from itertools import groupby
import itertools
import operator
from datetime import timedelta
import csv
import time

In [12]:
#gets eruption data from the HEK 
client = hek.HEKClient()
h2v = hek2vso.H2VClient()
tstart = '2013/10/01 00:00:00'
tend = '2013/10/31 23:59:59'
event_type = 'FE'
event_type_2 = 'AR'
eruptions = client.query(hek.attrs.Time(tstart,tend),hek.attrs.EventType(event_type))
active_region = client.query(hek.attrs.Time(tstart,tend),hek.attrs.EventType(event_type_2))
vso_records = h2v.translate_and_query(eruptions[10])



In [13]:
print len(active_region)

3382


In [14]:
 ###Plotting functions     
def TotalSunPlot(eruptions_good, eruptions_okay, trackno, filament):
    plt.grid(True)
    plt.ylabel('ycoord (arcsec)',fontsize=18)
    plt.xlabel('xcoord (arcsec)',fontsize=18)
    plt.axis([-1500, 1500, -1500, 1500]) 
    longitude2 = [elem["hpc_x"] for elem in eruptions_okay]
    latitude2 = [elem["hpc_y"] for elem in eruptions_okay]
    fig=plt.scatter(longitude2,latitude2,color='red',s=40)
    longitude3 = [elem["hpc_x"] for elem in eruptions_good]
    latitude3 = [elem["hpc_y"] for elem in eruptions_good]
    fig=plt.scatter(longitude3,latitude3,color='blue',s=50)
    Circle1 = plt.Circle((0,0), radius=1000, linestyle='solid', linewidth=2, fill=False)
    fig1 = plt.gcf()
    fig1.gca().add_artist(Circle1)
    fig1.set_size_inches(10.5,10.5)
    ax = fig1.add_subplot(111)
    ax.set_aspect('equal')
    ax.tick_params(axis='both', which='major', labelsize=14)
    plt.title('Filament Eruptions Intersect and are Contained in Filaments, October 2013')
    norm = mpl.colors.Normalize(vmin=0, vmax=280)
    cmap = plt.get_cmap('CMRmap')
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    scale= m.to_rgba(trackno)
    if trackno in close:
        cmap = plt.get_cmap('Reds')
        norm = mpl.colors.Normalize(vmin=5, vmax=270)
        m = cm.ScalarMappable(norm=norm, cmap=cmap)
        scale= m.to_rgba(trackno)
        plot_polygon=plt.Polygon(loads(filament).exterior,color=scale,linewidth=2,fill=None)
        fig1.gca().add_artist(plot_polygon)
    if trackno in contained: 
        norm = mpl.colors.Normalize(vmin=0, vmax=280)
        cmap = plt.get_cmap('winter')
        m = cm.ScalarMappable(norm=norm, cmap=cmap)
        scale= m.to_rgba(trackno)
        plot_polygon=plt.Polygon(loads(filament).exterior,color=scale,linewidth=2,fill=None)
        fig1.gca().add_artist(plot_polygon)                          
    box = [elem["hpc_bbox"] for elem in eruptions_okay]    #plotting only those eruptions boxes that intersect with filaments
    for elem in box:
        plot_box=plt.Polygon(loads(elem).exterior,color='red',linewidth=1,fill=None)
        fig1.gca().add_artist(plot_box)
    
def SetupSunPlotColor(eruptions, trackno, filament):
    plt.grid(True)
    plt.ylabel('ycoord (arcsec)',fontsize=18)
    plt.xlabel('xcoord (arcsec)',fontsize=18)
    plt.axis([-1500, 1500, -1500, 1500])
    cmap = plt.get_cmap('nipy_spectral')
    norm = mpl.colors.Normalize(vmin=0, vmax=280)
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    scale= m.to_rgba(trackno)
    longitude = [elem["hpc_x"] for elem in eruptions]  #plotting only those eruptions that are contained in filaments 
    latitude = [elem["hpc_y"] for elem in eruptions]   #plotting only those eruptions that are contained in filaments
    for x,y in zip(longitude, latitude):
        fig=plt.scatter(x, y, color=next(colors),s=50)
    Circle1 = plt.Circle((0,0), radius=1000, linestyle='solid', linewidth=2, fill=False)
    fig1 = plt.gcf()
    fig1.gca().add_artist(Circle1)
    fig1.set_size_inches(10.5,10.5)
    ax = fig1.add_subplot(111)
    ax.set_aspect('equal')
    ax.tick_params(axis='both', which='major', labelsize=14)
    plt.title('Filament Eruptions and Tracks on the Sun, October 2013')
    plot_polygon=plt.Polygon(loads(filament).exterior,color=scale,linewidth=1,fill=None)
    fig1.gca().add_artist(plot_polygon) 

       
def SetupSunPlotBox(eruptions, trackno, filament):
    plt.grid(True)
    plt.ylabel('ycoord (arcsec)',fontsize=18)
    plt.xlabel('xcoord (arcsec)',fontsize=18)
    plt.axis([-1500, 1500, -1500, 1500])
    longitude = [elem["hpc_x"] for elem in eruptions] #plotting only those eruptions that intersect with  filaments
    latitude = [elem["hpc_y"] for elem in eruptions]
    fig=plt.scatter(longitude,latitude,color='black',s=50) 
    Circle1 = plt.Circle((0,0), radius=1000, linestyle='solid', linewidth=2, fill=False)
    fig1 = plt.gcf()
    fig1.gca().add_artist(Circle1)
    fig1.set_size_inches(10.5,10.5)
    ax = fig1.add_subplot(111)
    ax.set_aspect('equal')
    ax.tick_params(axis='both', which='major', labelsize=14)
    plt.title('Filament Eruptions (Bounds) that Intersect Filaments, October 2013')
    box = [elem["hpc_bbox"] for elem in eruptions]    #plotting only those eruptions boxes that intersect with filaments
    for elem in box:
        plot_box=plt.Polygon(loads(elem).exterior,color='black',linewidth=1,fill=None)
        fig1.gca().add_artist(plot_box) 
    norm = mpl.colors.Normalize(vmin=0, vmax=280)
    cmap = plt.get_cmap('nipy_spectral')
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    scale= m.to_rgba(trackno) 
    plot_polygon=plt.Polygon(loads(filament).exterior,color=scale,linewidth=1,fill=None)
    fig1.gca().add_artist(plot_polygon)

        
 

    

In [15]:
##filters the eruptions so the data is unique
times=[elem["event_starttime"] for elem in eruptions] 
result=[]
indices=[]
print len(eruptions)
### the following loop will remove identical eruptions that have been recorded at different wavelengths by sorting with time
for element in times:
    if element not in result:
        indices.append((times.index(element)))
        result.append(element)
print indices  #important indices 
filtered=[eruptions[i] for i in indices] #creates a filtered list of objects that are not indentical

50
[0, 1, 2, 5, 6, 9, 13, 17, 21, 28, 29, 32, 33, 37, 40, 41, 43, 46, 47]


In [16]:
##filters the eruptions so the data is unique
times_ar=[elem["event_starttime"] for elem in active_region] 
result=[]
indices=[]
#print len(eruptions)
### the following loop will remove identical eruptions that have been recorded at different wavelengths by sorting with time
for element in times_ar:
    if element not in result:
        indices.append((times_ar.index(element)))
        result.append(element)
print indices  #important indices 
filtered_active=[active_region[i] for i in indices] #creates a filtered list of objects that are not indentical
print len(filtered_active)

[0, 8, 15, 26, 34, 42, 50, 59, 68, 77, 86, 95, 104, 105, 106, 117, 128, 141, 150, 160, 168, 178, 187, 197, 203, 204, 212, 216, 221, 222, 232, 233, 238, 250, 255, 266, 270, 281, 285, 286, 297, 304, 318, 323, 333, 338, 347, 352, 353, 363, 368, 378, 385, 395, 401, 417, 422, 433, 439, 449, 454, 455, 466, 472, 480, 484, 492, 497, 511, 515, 523, 528, 529, 538, 542, 543, 544, 555, 562, 572, 578, 588, 593, 607, 612, 623, 629, 640, 641, 648, 660, 667, 678, 687, 698, 704, 720, 726, 727, 738, 742, 743, 751, 754, 759, 770, 775, 785, 791, 801, 807, 826, 830, 840, 844, 855, 861, 862, 874, 882, 893, 900, 905, 923, 928, 937, 942, 951, 956, 965, 971, 981, 989, 998, 1003, 1020, 1021, 1027, 1038, 1039, 1045, 1056, 1061, 1072, 1078, 1089, 1097, 1108, 1116, 1134, 1141, 1151, 1152, 1160, 1170, 1179, 1189, 1199, 1208, 1215, 1216, 1226, 1234, 1250, 1257, 1267, 1275, 1285, 1292, 1302, 1310, 1319, 1327, 1334, 1349, 1359, 1367, 1375, 1376, 1384, 1392, 1400, 1409, 1415, 1426, 1427, 1434, 1443, 1462, 1470, 1471, 1

# The following kernel reads in the tracking document, groups the data by track number, and adds  "null" data for all tracks that have less than three entries. 

In [17]:
#filament track data
fi_tracked = pd.read_csv('FIDustinTracked_2013_10.txt',delimiter='\t')
fi_tracked.head()
tracks=fi_tracked['Track No.']  #pulls out hthe track numbers in list format 
good_tracks = fi_tracked.groupby('Track No.').filter(lambda x: len(x['Track No.']) >= 3, dropna=False) 




IOError: File FIDustinTracked_2013_10.txt does not exist

# The following kernel looks at both "good tracks" (tracks with 3 or more elements) and "filtered" eruption data (data that does not occur at multiple wavelengths). The code loops through the list of filtered eruptions and checks if any eruption is contained inside a good track (also iterated through with a loop). 

In [ ]:
margin = datetime.timedelta(hours=12)             
FT_time = good_tracks['HEK_time']    #Finds the time stamps for all the filaments
FE_time = [elem["event_starttime"] for elem in filtered] #Finds the time stamps for all the eruptions
FT_poly = good_tracks['HPC_BB_Poly']    #Finds the polygon coordinates for the good_tracks
FE_coords = [elem["hpc_coord"] for elem in filtered] #Finds the point coordinates for the filtered eruptions 
contained=[]    ####This will be a helpful list for finding which tracks have eruptions contained inthem 
loopnumb=[]    #to keep track of the eruption number 
practice=[]
for j in range(0, len(FE_coords)):    #looks at all possible eruptions 
    FET=(parse_time(FE_time[j])) 
    FE_point_obj = loads(FE_coords[j]) #creates a shapely point object
    for i in range(0, len(FT_poly)):  #looks at all possible tracks 
        if pd.isnull(FT_poly[i]):   #skips over tracks of less than 3 elements 
            pass
        else:   
            FLT=parse(FT_time[i]) 
            FT_poly_obj = loads(FT_poly[i])   #creates polygon object from track
            if FT_poly_obj.contains(FE_point_obj) and FET-margin <= FLT <= FET + margin: #checks if point is contained in polygon 
                contained.append(good_tracks['Track No.'][i])   #finds the track number of these tracks 
                loopnumb.append(j)
                print FE_time[j]
                if good_tracks['Track No.'][i] == 16.0:
                    practice.append((FE_time[j], good_tracks['Track No.'][i]))
                print "This means that the eruption point is contained in the filament polygon"
                print j,i  #the indicies of the eruption and filament track
erup=[indices[i] for i in loopnumb]   #getting the eruption index from a list of possible indices
filtered2=[eruptions[i] for i in erup]   #eruption data that correspond to being contained in a filament 
print erup

In [ ]:
good_polygons=[] #Trying to find polygon data for the filaments that contained eruptions and at a good time 
for i in range(len(good_tracks)): #iterating through the whole list of tracks 
    if good_tracks['Track No.'][i] in contained:  #if the track number is in the lists created previously 
         good_polygons.append((good_tracks['Track No.'][i],good_tracks['HPC_BB_Poly'][i]))#append the polygon data to good polygons

#Main plotting call right now. This should filter by containment/location and time

In [ ]:
####makes a colorscale for the eruptions
colorscale=[]
norm = mpl.colors.Normalize(vmin=0, vmax=280)
cmap = plt.get_cmap('nipy_spectral')
m = cm.ScalarMappable(norm=norm, cmap=cmap)
for item in contained: #plotting those filaments that have eruptions contained within them
    scale1= m.to_rgba(item) 
    colorscale.append(scale1)

####plots eruptions/sun 
plt.grid(True)
plt.ylabel('ycoord (arcsec)',fontsize=18)
plt.xlabel('xcoord (arcsec)',fontsize=18)
plt.axis([-1500, 1500, -1500, 1500])
longitude = [elem["hpc_x"]for elem in filtered2]
latitude = [elem["hpc_y"] for elem in filtered2]
fig=plt.scatter(longitude,latitude,c= colorscale,s=50)
Sun=plt.Circle((0,0), radius=1100, color='black', fill=False)
plt.gcf().gca().add_artist(Sun)
plt.title('Eruptions Contained in Filaments on the Sun, October 2013')
fig1 = plt.gcf()
fig1.set_size_inches(10.5,10.5)
ax = fig1.add_subplot(111)
ax.set_aspect('equal') 

####plots filament tracks 
for poly in good_polygons: 
    scale=m.to_rgba(poly[0])
    plot_polygon=plt.Polygon(loads(poly[1]).exterior,color=scale,linewidth=1,fill=None)
    fig1.gca().add_artist(plot_polygon)
plt.show()

# The following kernel looks at both "good tracks" (tracks with 3 or more elements) and "filtered" eruption data (data that does not occur at multiple wavelengths). The code loops through the list of filtered eruptions and checks if any eruption box intersects with a good track (also iterated through with a loop).

In [ ]:
margin = datetime.timedelta(hours=12)             
FT_time = good_tracks['HEK_time']    #Finds the time stamps for all the filaments
FE_time = [elem["event_starttime"] for elem in filtered] #Finds the time stamps for all the eruptions
FT_poly = good_tracks['HPC_BB_Poly'] 
FE_box = [elem["hpc_bbox"] for elem in filtered] 
FE_coords = [elem["hpc_coord"] for elem in filtered]
intersected=[] #the tracks that intersect with eruption boxes
event=[]  #eruption point 
manypoly=[] #center point of filament
erupno=[]   #eruption's number 
time_erup=[]
start = time.time()
########## to only plot certain eruptions
for j in range(0, len(FE_box)):    #looks at all possible eruptions 
    FET=(parse_time(FE_time[j]))
    FE_box_obj = loads(FE_box[j]) #creates a shapely box object object
    FE_point_obj=loads(FE_coords[j])
    for i in range(0, len(FT_poly)):  #looks at all possible tracks 
        if pd.isnull(FT_poly[i]):   #skips over tracks of less than 3 elements 
            pass
        else:    
            FLT=parse(FT_time[i])
            FT_poly_obj = loads(FT_poly[i])   #creates polygon object from track
            if FT_poly_obj.intersects(FE_box_obj) and FET-margin <= FLT <= FET + margin: #checks if box intersects with polygon
                intersected.append(good_tracks['Track No.'][i])   #finds the track number of these tracks 
                manypoly.append(loads(good_tracks['HPC_Center'][i]))  #grabs the center point coords of the filament 
                event.append(FE_point_obj)      #grabs the eruption point 
                erupno.append(j)
                time_erup.append(FET)
sec = time.time() - start
print sec%60, 's'
multi=zip(intersected, event, manypoly, erupno, time_erup) #creates a list of tuples of [(track no, eruption polygon point, filament polygon point, iteration).

# the following filters the "intersected" list of tracks further by finding those tracks with center point CLOSEST to the coordinates of each eruption. thus, the number of tracks should equal the number of eruptions 

In [ ]:
dist=[]  ###will be a list of distances between the eruptions and center of the filaments 
final_erup=[]   #eruption's number
practice=[]
for item in multi:
     dist.append(item[1].distance(item[2])) #finding the distance between the filament polygon center and the eruption 

##the following is Nicole's code using pandas
multi3=zip(intersected, event, dist, erupno, time_erup) #This is what you did
test = pd.DataFrame(multi3) #Turn multi3 into a pandas dataframe
columns=['Track No.','Eruption Coord','Distance to filament','Eruption No.', 'Eruption Time'] #Name the columns so they are easier to deal with
test.columns=columns
idx = test.groupby(['Track No.'])['Distance to filament'].transform(min) == test['Distance to filament']
only_min=test[idx]
only_min = only_min.sort('Track No.')
close=[]   #those tracks closest to the eruptions 
corresponding=[] ### just to know which tracks correspond to which eruption 
for index, item in only_min.iterrows():
    close.append(item[0])   #for the tracks 
    final_erup.append(item[3])   #for the eruptions 
    corresponding.append((item[0], item[3]))   # to see which tracks correspond to which eruptions
    practice.append((item[4], item[0]))
print corresponding   #filament track number, eruption number  
    
erupbox_with_track=[indices[i] for i in final_erup]
filtered3=[eruptions[i] for i in erupbox_with_track]

In [ ]:
okay_polygons=[] #Trying to find polygon data for the filaments that intersect with eruption box
for i in range(len(good_tracks)): #iterating through the whole list of tracks 
    if good_tracks['Track No.'][i] in close: 
            okay_polygons.append((good_tracks['Track No.'][i],good_tracks['HPC_BB_Poly'][i])) #append the polygon data to good polygons

# Plots filament tracks versus the eruptions on the surface of the Sun

In [ ]:
####makes a colorscale for eruptions 
colorscale2=[]
norm = mpl.colors.Normalize(vmin=0, vmax=280)
cmap = plt.get_cmap('nipy_spectral')
m = cm.ScalarMappable(norm=norm, cmap=cmap)
for item in close: #plotting those filaments that have eruptions contained within them
    scaleR= m.to_rgba(item) 
    colorscale2.append(scaleR)
    
####plots eruptions/sun
plt.grid(True)
plt.ylabel('ycoord (arcsec)',fontsize=18)
plt.xlabel('xcoord (arcsec)',fontsize=18)
plt.axis([-1500, 1500, -1500, 1500])
longitude = [elem["hpc_x"]for elem in filtered3]
latitude = [elem["hpc_y"] for elem in filtered3]
fig=plt.scatter(longitude,latitude,c= colorscale2,s=50)
Sun=plt.Circle((0,0), radius=1100, color='black', fill=False)
plt.gcf().gca().add_artist(Sun)
plt.title('Filament Eruptions on the Sun October 2013')
fig1 = plt.gcf()
fig1.set_size_inches(10.5,10.5)
ax = fig1.add_subplot(111)
ax.set_aspect('equal')

####plots eruption boxes 
box = [elem["hpc_bbox"] for elem in filtered3]    #plotting only those eruptions boxes that intersect with filaments
boxinfo=zip(box, close)
for item in boxinfo:
        scaleR= m.to_rgba(item[1])
        plot_box=plt.Polygon(loads(item[0]).exterior,color=scaleR,linewidth=1,fill=None)
        fig1.gca().add_artist(plot_box)
        
###plots filament tracks         
for poly in okay_polygons: #plotting those filaments that intersect the bounding box of eruptions
    scale2=m.to_rgba(poly[0])
    plot_polygon=plt.Polygon(loads(poly[1]).exterior,color=scale2,linewidth=1,fill=None)
    fig1.gca().add_artist(plot_polygon)
plt.show()

# Plots each track seperately versus the eruptions

In [ ]:
   ###new updated eruption list for each track 
tot=[]
unique=[]
erupind=[]
close_unique=np.unique(close)
for tracks in close_unique:
    tot.append(unique)
    unique=[]
    for i in range(0, len(good_tracks)):
        if tracks == good_tracks['Track No.'][i]:
            unique.append(i)
            
for item in tot:
    colorscale2=[]
    norm = mpl.colors.Normalize(vmin=0, vmax=280)
    cmap = plt.get_cmap('nipy_spectral')
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    for thing1 in close: #plotting those filaments that have eruptions contained within them
        scaleR= m.to_rgba(thing1) 
        colorscale2.append(scaleR)
####plots eruptions/sun
    plt.grid(True)
    plt.ylabel('ycoord (arcsec)',fontsize=18)
    plt.xlabel('xcoord (arcsec)',fontsize=18)
    plt.axis([-1500, 1500, -1500, 1500])
    #filtered4=[eruptions[i] for i in erupind]
    #longitude = [elem["hpc_x"]for elem in filtered3]
    #latitude = [elem["hpc_y"] for elem in filtered3]
    #fig=plt.scatter(longitude,latitude, c = colorscale2,s=50)
    fig1 = plt.gcf()
    Sun=plt.Circle((0,0), radius=1100, color='black', fill=False)
    plt.gcf().gca().add_artist(Sun)
    fig1.set_size_inches(10.5,10.5)
    ax = fig1.add_subplot(111)
    ax.set_aspect('equal')
        ##############################
    for i in range(0, len(item)):
        #filtered4=[]
        j=item[i]
        trnum=  good_tracks['Track No.'][j]
        stime=  good_tracks['HEK_time'][j]
        for thing1 in corresponding:
            if thing1[0] == trnum:
                erupind.append(thing1[1])
        verup=[indices[i] for i in erupind]   #getting the eruption index from a list of possible indices
        erupind=[]
        filtered4=[eruptions[k] for k in verup]
        longitude = [elem["hpc_x"]for elem in filtered4]
        latitude = [elem["hpc_y"] for elem in filtered4]
        erup_t = [elem["event_starttime"] for elem in filtered4]
        for thing2 in erup_t:
            tim=(parse_time(thing2))
            print tim #to see the time of eac filament polygon 
        plt.text(1400,1400,str(erup_t))
        fig=plt.scatter(longitude,latitude, c = 'grey',s=50)
        scale2=m.to_rgba(good_tracks['Track No.'][j])
        plot_polygon=plt.Polygon(loads(good_tracks['HPC_BB_Poly'][j]).exterior,color=scale2,linewidth=1,fill=None)
        fig1.gca().add_artist(plot_polygon)
        plt.title('Track Number %(Trackno)d at %(starttime)s on the Sun.' % {"Trackno" : trnum, "starttime": stime})
    plt.show()


# The following code should only be run AFTER the erupting filaments found with the algorithm are confirmed to erupt via Jhelioviewer, or some other visualiztion

In [ ]:
#tracknumbers_oct_2013=[]
#good_track_no_oct_2013=[]
#with open('Oct2013Track_eruptions.csv', 'rb') as f:    ###opens the excel document containing confirmed eruption track information
    #reader = csv.reader(f)
    #vdata = [row for row in csv.reader(f.read().splitlines())]   #splits the document by line
#for row in vdata:
    #tracknumbers_oct_2013.append(row[0])      ###appends just the track number
#for i in range(0, 2):
    #tracknumbers_oct_2013.pop(i)     #removes heading text for file 

#tracknumbers_oct_2013.remove('')
#for item in tracknumbers_oct_2013:      
    #x=float(item)
    #good_track_no_oct_2013.append(x)  #appends the track numbers as a float to a new list    #list of erupting tracks in workable format

# The next two kernels use the list "practice" which contains track number and associated eruption time, to make a new list called "step" which contains track number, polygon length, eruption time, and track time. then, they are plotted in a series to show how the length of each filament changes before an eruption 


In [ ]:
print good_track_no_oct_2013 #this a manual input list of all tracks that correlated to eurptions (visually)
find=list(set(close) & set(good_track_no_oct_2013))    #finds the intersection of tracks we have data for, and tracks that erupted 
step=[]
bad=[]
for p in practice:  #practice has track number associated with eruption time  
    for tracks in find:
        for i in range(0, len(good_tracks)):
            if good_tracks['Track No.'][i] == p[1]:   ##checks for the right eruption time 
                if good_tracks['Track No.'][i] == tracks:  ### next line should append time of each event, the trakc number, and the length of each polygon
                    if [tracks, parse(good_tracks['HEK_time'][i]), good_tracks['fi_length'][i], parse_time(p[0])] not in step:
                        step.append([tracks, parse(good_tracks['HEK_time'][i]), good_tracks['fi_length'][i], parse_time(p[0])])
for it in step:
    print it[0], it[3]

In [ ]:
from matplotlib.dates import WeekdayLocator
colorscale2=[]
norm = mpl.colors.Normalize(vmin=0, vmax=280)
cmap = plt.get_cmap('nipy_spectral')
m = cm.ScalarMappable(norm=norm, cmap=cmap)
interest=[]
x=[]
y=[]
t=[]
for track in find:
    scale3=m.to_rgba(track)
    x=[] ### will be time data
    y=[] ## will be length data 
    r=[]
    t=[]
    interest=[]
    emp=[]
    for r in step:      #iterates through the data 
        if r[0] == track:  #finds the correct track
            if r not in interest:
                 interest.append(r)   #appends the data to a new list
    for k in range(0, len(interest)):
        t2= interest[k][3]
        t1=interest[k][1]
        t3=t2-t1
        t.append(t3.seconds)
        x.append(t1)
        y.append(interest[k][2])
    plt.plot_date(x, y, xdate=True,ydate=False, color=scale3, linestyle='-')
    plt.axvline(interest[k][3], linewidth=2, color=scale3, linestyle='--', label='eruption')
    plt.text(interest[k][3], 28000000000,'Erup for Tr. %(Trackno)d' %{"Trackno": track},rotation=80)
    #ax.xaxis.set_major_locator(AutoDateLocator())
    plt.title('Filament length (Track) versus eruption time')
    plt.grid(True)
    fig1 = plt.gcf()
    fig1.set_size_inches(10.5,10.5)
    plt.xticks( rotation= 80 )
    plt.ylabel('Filament length (cm)')
plt.show()

In [ ]:
colorscale2=[]
norm = mpl.colors.Normalize(vmin=0, vmax=280)
cmap = plt.get_cmap('nipy_spectral')
m = cm.ScalarMappable(norm=norm, cmap=cmap)
interest=[]
x=[]
y=[]
t=[]
for track in find:
    scale3=m.to_rgba(track)
    x=[] ### will be time data
    y=[] ## will be length data 
    r=[]
    t=[]
    interest=[]
    emp=[]
    for r in step:      #iterates through the data 
        if r[0] == track:  #finds the correct track
            if r not in interest:
                 interest.append(r)   #appends the data to a new list
    for k in range(0, len(interest)):
        t2= interest[k][3]    ##eruption time
        t1=interest[k][1]    #filament time
        t3=timedelta.total_seconds(t2-t1) #difference between the two in seconds
        t.append(int(t3))
        y.append(interest[k][2])
    plt.plot(t, y, color=scale3)
    plt.axvline(0, linewidth=3, color='k', linestyle='--', label='Eruption')
    plt.title('Filament length (Track) versus eruption time')
    plt.grid(True)
    fig1 = plt.gcf()
    fig1.set_size_inches(10.5,10.5)
    plt.xlabel('Filament Track time (s) from time of associated eruption at 0')
    plt.ylabel('Filament length (cm)')
plt.show()

In [ ]:
####finding the time of specific eruptions
index1=[12]
test=[filtered[i] for i in index1]
timeerup = [elem['event_starttime'] for elem in test]
print timeerup

In [ ]:
#### to check tracks that aren't "good" for some reason 
noerup=[]
check_track1=[166.0]
check_poly1=[]
for i in range(len(good_tracks)): #iterating through the whole list of tracks 
    #if good_tracks['Track No.'][i] in intersected and good_tracks['Track No.'][i] in okay_filaments:   #if the track number is in the list created previously 
    if fi_tracked['Track No.'][i] in check_track1: 
            check_poly1.append((fi_tracked['Track No.'][i],fi_tracked['HPC_BB_Poly'][i])) #append the polygon data to good polygons
            print fi_tracked['Track No.'][i],fi_tracked['HEK_time'][i]
for poly in check_poly1: #plotting those filaments that have eruptions contained within them
    SetupSunPlotColor(noerup, poly[0], poly[1]) #filament eruption coordinates, track number, track polygon coordinates



# finding more transiting tracks

In [ ]:
#### just for fun, trying to find tracks that transit the surface
fi_tracked = pd.read_csv('FIDustinTracked_2013_10.txt',delimiter='\t')
long_tracks = fi_tracked.groupby('Track No.').filter(lambda x: len(x['Track No.']) >= 4, dropna=False) 

In [ ]:
#uploads all track numbers of filaments that have been CONFIRMED to erupt
tracknumbers_oct_2013=[]
good_track_no_oct_2013=[]
erup_info_oct_2013=[]
with open('Oct2013Track_eruptions.csv', 'rb') as f:    ###opens the excel document containing confirmed eruption track information
    reader = csv.reader(f)
    vdata = [row for row in csv.reader(f.read().splitlines())]   #splits the document by line
for row in vdata:
    tracknumbers_oct_2013.append(row[0])      ###appends just the track number
    erup_info_oct_2013.append((row[0], row[7]))
for i in range(0, 2):
    tracknumbers_oct_2013.pop(i)     #removes heading text for file 
    erup_info_oct_2013.pop(i)
######
del tracknumbers_oct_2013[0]
del erup_info_oct_2013[0]
##########
for item in tracknumbers_oct_2013:
    if item != '':
        x=float(item)
        good_track_no_oct_2013.append(x)  #appends the track numbers as a float to a new list 

In [ ]:
#uploads all track numbers of filaments that already been documented as non-erupting 
tracknumbers_oct_2013_trans=[]
good_track_no_oct_2013_trans=[]
with open('Transit_tracks_oct_2013.csv', 'rb') as f:    ###opens the excel document containing confirmed eruption track information
    reader = csv.reader(f)
    vdata = [row for row in csv.reader(f.read().splitlines())]   #splits the document by line
for row in vdata:
    tracknumbers_oct_2013_trans.append(row[0])      ###appends just the track number
for i in range(0, 2):
    tracknumbers_oct_2013_trans.pop(i)     #removes heading text for file 
del tracknumbers_oct_2013_trans[0]
##########
for item in tracknumbers_oct_2013_trans:  
    if item != '':
        x=float(item)
        good_track_no_oct_2013_trans.append(x)  #appends the track numbers as a float to a new list 
print good_track_no_oct_2013_trans

In [ ]:
#the following loop does a similar thing to the "filter" process to correlate eruptions to filaments except 
#the reverse to find filaments that are NOT already documented and NOT near in time/space any eruption
margin = datetime.timedelta(hours=12)  
FT_track = long_tracks['Track No.']
FT_point = long_tracks['HPC_Center']
FT_time = long_tracks['HEK_time']    #Finds the time stamps for all the filaments
FE_time = [elem["event_starttime"] for elem in filtered] #Finds the time stamps for all the eruptions
FT_poly = long_tracks['HPC_BB_Poly'] 
FE_box = [elem["hpc_bbox"] for elem in filtered] 
FE_coords = [elem["hpc_coord"] for elem in filtered] 
non_erup=[] #the tracks that intersect with eruption boxes
yes_erup=[]
for i in range(0, len(FT_poly)):
    if pd.isnull(FT_poly[i]):    #any tracks less than 4 elements 
        pass
    else:
        if FT_track[i] in good_track_no_oct_2013_trans or FT_track[i] in good_track_no_oct_2013:  #checks if track is already documented 
            pass
        else:
            FLT = parse(FT_time[i])
            FT_poly_obj = loads(FT_poly[i])
            for j in range(0, len(FE_box)):
                FET=(parse_time(FE_time[j]))
                FE_box_obj = loads(FE_box[j]) #creates a shapely box object object
                FE_point_obj=loads(FE_coords[j])
                if FT_poly_obj.intersects(FE_box_obj) and FET-margin <= FLT <= FET + margin: #checks if track is close in time and space to eruption
                    yes_erup.append(FT_track[i])
                    pass   ####for some reason this isn't working how I expect it to
                else:
                    non_erup.append(FT_track[i])    ####appending the tracks that made it through the filters

non_erup = np.unique(non_erup)
good_ind=[]
for i in range(0, len(non_erup)):
    if non_erup[i] not in yes_erup:
        good_ind.append(i)
non_erup_filt = [non_erup[j] for j in good_ind]
print non_erup_filt

In [ ]:
 ###new updated eruption list for each track 
tot=[]
unique=[]
erupind=[]
for tracks in non_erup_filt:
    tot.append(unique)
    unique=[]
    for i in range(0, len(good_tracks)):
        if tracks == good_tracks['Track No.'][i]:
            unique.append(i)
            
for item in tot:
    colorscale2=[]
    norm = mpl.colors.Normalize(vmin=0, vmax=3000)
    cmap = plt.get_cmap('nipy_spectral')
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    for thing1 in non_erup_filt: #plotting those filaments that have eruptions contained within them
        scaleR= m.to_rgba(thing1) 
        colorscale2.append(scaleR)
####plots eruptions/sun
    plt.grid(True)
    plt.ylabel('ycoord (arcsec)',fontsize=18)
    plt.xlabel('xcoord (arcsec)',fontsize=18)
    plt.axis([-1500, 1500, -1500, 1500])
    fig1 = plt.gcf()
    Sun=plt.Circle((0,0), radius=1100, color='black', fill=False)
    plt.gcf().gca().add_artist(Sun)
    fig1.set_size_inches(10.5,10.5)
    ax = fig1.add_subplot(111)
    ax.set_aspect('equal')
        ##############################
    for i in range(0, len(item)):
        j=item[i]
        trnum=  good_tracks['Track No.'][j]
        stime=  good_tracks['HEK_time'][j]
        print stime
        for thing1 in corresponding:
            if thing1[0] == trnum:
                erupind=[]
                #erupind.append(thing1[1])
        verup=[indices[i] for i in erupind]   #getting the eruption index from a list of possible indices
        erupind=[]
        filtered4=[eruptions[k] for k in verup]
        longitude = [elem["hpc_x"]for elem in filtered4]
        latitude = [elem["hpc_y"] for elem in filtered4]
        erup_t = [elem["event_starttime"] for elem in filtered4]
        for thing2 in erup_t:
            tim = (parse_time(thing2))
        plt.text(1400,1400,str(erup_t))
        fig=plt.scatter(longitude,latitude, c = 'grey',s=50)
        scale2=m.to_rgba(good_tracks['Track No.'][j])
        plot_polygon=plt.Polygon(loads(good_tracks['HPC_BB_Poly'][j]).exterior,color=scale2,linewidth=1,fill=None)
        fig1.gca().add_artist(plot_polygon)
        plt.title('Track Number %(Trackno)d at %(starttime)s on the Sun.' % {"Trackno" : trnum, "starttime": stime})
    plt.show()